In [1]:
resource = "../../data/generated/"
results = "../../data/generated/"

In [2]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from linearmodels.panel import PanelOLS, compare
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from gmm import *
from linearmodels.asset_pricing import LinearFactorModelGMM
from tabprintin.beautify import *
from statsmodels.sandbox.regression.gmm import GMM
from numpy import hstack, zeros, ones, array, mat, tile, reshape, squeeze, eye, asmatrix, column_stack, roll, concatenate

In [3]:
# Set the start and end dates of the analysis period
# full sample
sdatefull = pd.to_datetime('1975-01-01')
edatefull = pd.to_datetime('2021-12-31')
# first subsample
sdate1 = pd.to_datetime('1975-01-01')
edate1 = pd.to_datetime('2007-11-30')
# second subsample
sdate2 = pd.to_datetime('2007-12-01')
edate2 = pd.to_datetime('2021-12-31')

sdate = [sdatefull, sdate1, sdate2]
edate = [edatefull, edate1, edate2]

In [4]:
ts = pd.read_csv(resource + 'time_series.csv', parse_dates=['date'], index_col=['date'])
ts.index.freq = 'M'

# Compute the log change of industrial production over next 12 months (or just growth)
# ts['log_indprod_growth_nextyear'] = np.log(ts['ind_prod'].shift(-12) / ts['ind_prod'].shift(-1)) #wrong
ts['log_indprod_growth_nextyear'] = np.log(ts['ind_prod'].shift(-12) / ts['ind_prod'])
# ts['indprod_growth_nextyear'] = ts['ind_prod'].shift(-12) / ts['ind_prod'].shift(-1) - 1

##################
## Base assets ###
##################

# Market return
# ts['ex_mkt'] = ts['ex_mkt'] /100
ts['lag_ex_mkt'] = ts['ex_mkt'].shift(1)

# Compute the excess return of the long-term government bond portfolio (bloomberg data)
# ts['ex_long_gov_ret'] = ts['long_gov_ret'] - ts['rf']
# ts['lag_ex_long_gov_ret'] = ts['ex_long_gov_ret'].shift(1)

# Compute the excess return of the 10 year treasury government bond
ts['ex_b10ret'] = ts['b10ret'] - ts['rf']
ts['lag_ex_b10ret'] = ts['ex_b10ret'].shift(1)

# Compute the excess return of the intermediate-term government bond portfolio (bloomberg data)
# ts['ex_medium_gov_ret'] = ts['medium_gov_ret'] - ts['rf']
# ts['lag_ex_medium_gov_ret'] = ts['ex_medium_gov_ret'].shift(1)

# Compute the excess return of the 5 year treasury government bond
ts['ex_b5ret'] = ts['b5ret'] - ts['rf']
ts['lag_ex_b5ret'] = ts['ex_b5ret'].shift(1)

# Compute the excess return of the high-yield bond portfolio
ts['ex_high_yd_bd_ret'] = ts['high_yd_bd_ret'] - ts['rf']
ts['lag_ex_high_yd_bd_ret'] = ts['ex_high_yd_bd_ret'].shift(1)

# Compute the return for gold index
ts['ex_gold_ret'] = ts['gold'].pct_change() - ts['rf']
ts['lag_ex_gold_ret'] = ts['ex_gold_ret'].shift(1)

# Create dummies for 1987 (stock market crash), 1996-2002 (Internet bubble period), and 2007-2009 (financial crises)
ts['dummy_87'] = ((ts.index >= '1987-04-01') & (ts.index <= '1988-04-01')).astype(int)
ts['dummy_96_02'] = ((ts.index.year >= 1996) & (ts.index.year <= 2002)).astype(int)
ts['dummy_07_09'] = ((ts.index >= '2007-12-01') & (ts.index <= '2009-06-01')).astype(int)
# ts['dummy_07_09'] = ((ts.index.year >= 2007) & (ts.index.year <= 2009)).astype(int)

# Interactions
ts['slope_ex_mkt_87'] = ts['ex_mkt'] * ts['dummy_87']
ts['slope_ex_mkt_9602'] = ts['ex_mkt'] * ts['dummy_96_02']
ts['slope_ex_mkt_0709'] = ts['ex_mkt'] * ts['dummy_07_09']
ts['lag_slope_ex_mkt_87'] = ts['slope_ex_mkt_87'].shift(1)
ts['lag_slope_ex_mkt_9602'] = ts['slope_ex_mkt_9602'].shift(1)
ts['lag_slope_ex_mkt_0709'] = ts['slope_ex_mkt_0709'].shift(1)

#########################
### Control Variables ###
#########################
# Compute the 10 year minus 3 month government bond yield
ts['lag_10y_3m_gov_bd_yd'] = (ts['DGS10'] - ts['DTB3']).shift(1)

# Compute the 1 year minus 3 month government bond yield
ts['lag_1y_3m_gov_bd_yd'] = (ts['DGS1'] - ts['DTB3']).shift(1)

# Baa minus Aaa corporate bond yield
ts['lag_Baa_Aaa_bd_yd'] = (ts['BAA'] - ts['AAA']).shift(1)

# Compute the dividend yield on the S&P 500 index
ts['lag_sp_div_yd'] = (ts['sp_div'] / ts['sp_price']).shift(1)

# Compute the log change of industrial production over last 12 months (or just growth)
ts['log_indprod_growth_lastyear'] = np.log(ts['ind_prod'].shift(13) / ts['ind_prod'].shift(1))
# ts['indprod_growth_lastyear'] = ts['ind_prod'].shift(13) / ts['ind_prod'].shift(1)  - 1

# Compute the inflation over last 12 months
# ts['infl_lastyear'] = (ts['cpi'].shift(1) - ts['cpi'].shift(13)) / ts['cpi'].shift(13)
ts['infl_lastyear'] = np.log(ts['cpi'].shift(1) / ts['cpi'].shift(13))

# Compute the market portfolio excess return over last 12 months
ts['ex_mkt_lastyear'] = (((ts['ex_mkt'] + 100)/100).rolling(13).apply(lambda x: x[:-1].prod()) - 1) * 100

In [5]:
mimick_list = []
for i in range(len(sdate)):
    # Set analysis period
    ts_mimick = ts.loc[(ts.index >= sdate[i]) & (ts.index <= edate[i])]

    # Estimation
    regress = 'log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+slope_ex_mkt_0709+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602+lag_slope_ex_mkt_0709'
    if (ts_mimick['slope_ex_mkt_87'] == 0).all():
        regress = regress.replace("+slope_ex_mkt_87", "")
        regress = regress.replace("+lag_slope_ex_mkt_87", "")
    if (ts_mimick['slope_ex_mkt_9602'] == 0).all():
        regress = regress.replace("+slope_ex_mkt_9602", "")
        regress = regress.replace("+lag_slope_ex_mkt_9602", "")
    if (ts_mimick['slope_ex_mkt_0709'] == 0).all():
        regress = regress.replace("+slope_ex_mkt_0709", "")
        regress = regress.replace("+lag_slope_ex_mkt_0709", "")
    # mod = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+slope_ex_mkt_0709+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602+lag_slope_ex_mkt_0709',
    #             data=ts_mimick)
    mod = smf.ols(regress,
                data=ts_mimick)
    mimickfit = mod.fit(cov_type='HAC',cov_kwds={'maxlags':11})

    # Append results
    mimick_list.append(mimickfit)

%store mimick_list

for mimick in mimick_list:
    print(mimick.summary())

# tsfull = ts.loc[(ts.index >= sdatefull) & (ts.index <= edatefull)]
# ts1 = ts.loc[(ts.index >= sdate1) & (ts.index <= edate1)]
# ts2 = ts.loc[(ts.index >= sdate2) & (ts.index <= edate2)]

# with Bloomberg bond portfolio return data
# mod1 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_long_gov_ret+ex_medium_gov_ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear',
#                 data=ts1)

# mod2 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_long_gov_ret+ex_medium_gov_ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_long_gov_ret+lag_ex_medium_gov_ret+lag_ex_high_yd_bd_ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602',
#                 data=ts1)

# mod1 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear',
#                 data=ts1)

# mod2 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_high_yd_bd_ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602',
#                 data=ts1)

# mod1 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear',
#                 data=ts1)

# modfull = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+slope_ex_mkt_0709+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602+lag_slope_ex_mkt_0709',
#                 data=tsfull)
# mod1 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+slope_ex_mkt_0709+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602+lag_slope_ex_mkt_0709',
#                 data=ts1)
# mod2 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_b10ret+ex_b5ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+slope_ex_mkt_0709+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_b10ret+lag_ex_b5ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602+lag_slope_ex_mkt_0709',
#                 data=ts2)

# mimickfull = modfull.fit(cov_type='HAC',cov_kwds={'maxlags':11})
# mimick1 = mod1.fit(cov_type='HAC',cov_kwds={'maxlags':11})
# mimick2 = mod2.fit(cov_type='HAC',cov_kwds={'maxlags':11})

# %store mimickfull
# %store mimick1
# %store mimick2

# mimick_list = [mimickfull, mimick1, mimick2]

# for mimick in mimick_list:
#     print(mimick.summary())

Stored 'mimick_list' (list)
                                 OLS Regression Results                                
Dep. Variable:     log_indprod_growth_nextyear   R-squared:                       0.307
Model:                                     OLS   Adj. R-squared:                  0.279
Method:                          Least Squares   F-statistic:                     5.314
Date:                         Tue, 30 May 2023   Prob (F-statistic):           2.76e-13
Time:                                 10:31:53   Log-Likelihood:                 1060.9
No. Observations:                          564   AIC:                            -2076.
Df Residuals:                              541   BIC:                            -1976.
Df Model:                                   22                                         
Covariance Type:                           HAC                                         
                                  coef    std err          z      P>|z|      [0.025      0.9

Summary statistics

In [6]:
for i in range(len(sdate)):
    coef = mimick_list[i].params
    # ts[f'myp{i}'] = coef['ex_mkt'] * ts['ex_mkt'] + coef['ex_b10ret'] * ts['ex_b10ret'] + coef['ex_b5ret'] * ts['ex_b5ret'] + coef['ex_gold_ret'] * ts['ex_gold_ret'] + coef['slope_ex_mkt_87'] * ts['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * ts['slope_ex_mkt_9602'] + coef['slope_ex_mkt_0709'] * ts['slope_ex_mkt_0709']
    ts[f'myp{i}'] = (
        coef['ex_mkt'] * ts['ex_mkt']
        + coef['ex_b10ret'] * ts['ex_b10ret']
        + coef['ex_b5ret'] * ts['ex_b5ret']
        + coef['ex_gold_ret'] * ts['ex_gold_ret']
    )
    
    if 'slope_ex_mkt_87' in coef:
        ts[f'myp{i}'] += coef['slope_ex_mkt_87'] * ts['slope_ex_mkt_87']

    if 'slope_ex_mkt_9602' in coef:
        ts[f'myp{i}'] += coef['slope_ex_mkt_9602'] * ts['slope_ex_mkt_9602']

    if 'slope_ex_mkt_0709' in coef:
        ts[f'myp{i}'] += coef['slope_ex_mkt_0709'] * ts['slope_ex_mkt_0709']

# Get the unexpected inflation factor
ts['infl'] = np.log(ts['cpi'] / ts['cpi'].shift(1))
ts['delta_infl'] = ts['infl'] - ts['infl'].shift(1)
inf_ma1 = sm.tsa.arima.ARIMA(ts['delta_infl'], order=(0,0,1)).fit()
ts['fit_delta_infl'] = inf_ma1.fittedvalues
ts['ui'] = ts['delta_infl'] - ts['fit_delta_infl']

# Get the change in the aggregate survival probability factor (with my dsv)
mod3 = smf.ols('v_dsv ~ 1 + my_dsv', data=ts).fit()
parm = mod3.params
ts['fit_dsv'] = parm['Intercept'] + parm['my_dsv'] * ts['my_dsv']
ts['dsv'] = np.where(ts['v_dsv'].notna(), ts['v_dsv'], ts['fit_dsv'])

# Get the change in the aggregate survival probability factor (with dsv from CRI)
# mod3 = smf.ols('v_dsv ~ 1 + cri_dsv', data=ts).fit()
# parm = mod3.params
# ts['fit_dsv'] = parm['Intercept'] + parm['cri_dsv'] * ts['cri_dsv']
# ts['dsv'] = np.where(ts['v_dsv'].notna(), ts['v_dsv'], ts['fit_dsv'])

# Get the change in the average level of the term structure factor
ts['mean_term_structure'] = ts[['DTB3', 'DGS10']].mean(axis=1)
ts['ats'] = ts['mean_term_structure'] - ts['mean_term_structure'].shift(1)

# Get the change in the slope of the term strucutre factor
ts['diff_term_structure'] = ts['DGS10'] - ts['DTB3']
ts['sts'] = ts['diff_term_structure'] - ts['diff_term_structure'].shift(1)

# Get the change in the multilateral US dollar exchange rate factor
mod4 = smf.ols('TWEXM ~ 1 + DTWEXAFEGS', data=ts).fit()
parm = mod4.params
ts['fit_TWEXM'] = parm['Intercept'] + parm['DTWEXAFEGS'] * ts['DTWEXAFEGS']
ts['exchange_rate'] = np.where(ts['TWEXM'].notna(), ts['TWEXM'], ts['fit_TWEXM'])
ts['fx'] = ts['exchange_rate'] - ts['exchange_rate'].shift(1)
# ts['fx'] = np.log(ts['exchange_rate'] / ts['exchange_rate'].shift(1))

%store ts


Stored 'ts' (DataFrame)


In [15]:
summary_list = []
for i in range(len(sdate)):
    # Set analysis period
    ts_var = ts.loc[(ts.index >= sdate[i]) & (ts.index <= edate[i])]
    # summary = ts_var[[f'myp{i}','ui','dsv','ats','sts','fx']].describe().T  
    columns = [f'myp{i}', 'ui', 'dsv', 'ats', 'sts', 'fx']
    summary = pd.DataFrame(index=columns)
    summary['N'] = ts_var[columns].count()
    summary['mean'] = ts_var[columns].mean()
    summary['median'] = ts_var[columns].median()
    summary['std'] = ts_var[columns].std()
    summary['skew'] = ts_var[columns].skew()
    summary['kurt'] = ts_var[columns].kurtosis()
    summary['max'] = ts_var[columns].max()
    summary['min'] = ts_var[columns].min()
    summary_list.append(summary)

%store summary_list

Stored 'summary_list' (list)


Correlation

In [25]:
corr_list = []
for i in range(len(sdate)):
    # Set analysis period
    ts_corr = ts.loc[(ts.index >= sdate[i]) & (ts.index <= edate[i])]
    columns = ['ex_mkt', 'hml', 'smb', 'rmw', 'cma', 'mom',f'myp{i}', 'ui', 'dsv', 'ats', 'sts', 'fx']
    # Compute correlation between the variables in columns
    corr = ts_corr[columns].corr()
    corr_list.append(corr)

%store corr_list

Stored 'corr_list' (list)


Granger causality

In [19]:
mimick_colnames = ['log_indprod_growth_nextyear',
                    'ex_mkt',
                   'ex_b10ret',
                   'ex_b5ret',
                #    'ex_high_yd_bd_ret',
                   'ex_gold_ret', 
                   'slope_ex_mkt_87', 
                   'slope_ex_mkt_9602', 
                   'slope_ex_mkt_0709', 
                   'rf', 
                   'lag_10y_3m_gov_bd_yd',
                   'lag_1y_3m_gov_bd_yd',
                   'lag_Baa_Aaa_bd_yd',
                   'lag_sp_div_yd',
                   'log_indprod_growth_lastyear',
                   'infl_lastyear',
                   'ex_mkt_lastyear',
                   'lag_ex_mkt',
                   'lag_ex_b10ret',
                   'lag_ex_b5ret',
                #    'lag_ex_high_yd_bd_ret',
                   'lag_ex_gold_ret',
                   'lag_slope_ex_mkt_87',
                   'lag_slope_ex_mkt_9602',
                   'lag_slope_ex_mkt_0709'
                   ]

var_exogcolnames = ['hml','smb','rmw','cma','mom','ui','dsv','ats','sts','fx']

In [20]:
class gmmvar(GMM):
    def momcond(self, params):
        global nbase
        var_exog = self.exog
        y = self.endog
        mimick = self.instrument
        coeff = squeeze(array(params)) 
        
        # first stage
        mimick_endog = mimick[:,0]
        mimick_exog = mimick[:,1:]
        mimick_exog = sm.add_constant(mimick_exog)
        T,K = mimick_exog.shape
        mimick_exog = reshape(mimick_exog,(T,K))
        mimick_endog = reshape(mimick_endog,(T,1))
        mimick_coeff = coeff[:K]
        mimick_coeff = reshape(mimick_coeff,(K,1))
        mimick_err = (mimick_endog - mimick_exog @ mimick_coeff)
        moments1 = mimick_exog * kron(mimick_err,ones((1,K)))
        moments1 = moments1[1:] # delete the first row

        # compute myp
        myp = mimick[:, 1:(nbase+1)] @ mimick_coeff[1:(nbase+1)] 

        # var
        var_exog = hstack((var_exog[:, :5], myp, var_exog[:, 5:]))
        var_lag = var_exog[:-1]
        var_lag = sm.add_constant(var_lag)
        var_exog = var_exog[1:]
        T,M = var_exog.shape
        T,N = var_lag.shape
        var_exog = reshape(var_exog,(T,M))
        var_lag = reshape(var_lag,(T,N))
        var_coeff = params[K:]
        var_coeff = reshape(var_coeff,(N,M))
        var_err = (var_exog - var_lag @ var_coeff)
        moments2 = tile(var_lag, (1, M)) * kron(var_err,ones((1,N)))
        return column_stack((moments1,moments2))

In [21]:
gmmvar_list = []
for i in range(len(sdate)):
    # Set analysis period
    ts_var = ts.loc[(ts.index >= sdate[i] + MonthBegin(-1)) & (ts.index <= edate[i])]

    # Var (without gmm)
    var = VAR(ts_var[['hml','smb','rmw','cma','mom',f'myp{i}','ui','dsv','ats','sts','fx']]).fit(maxlags=1, trend='c')
    # print(var.summary())

    nbase = 7
    mimick_colnames_sample = mimick_colnames
    if (ts_var['slope_ex_mkt_87'] == 0).all():
        mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
        nbase -= 1
    if (ts_var['slope_ex_mkt_9602'] == 0).all():
        mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
        nbase -= 1
    if (ts_var['slope_ex_mkt_0709'] == 0).all():
        mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_0709' not in colname]
        nbase -= 1

    # Set mimicking portfolio data
    mimick_cols = ts_var[mimick_colnames_sample].values
    T,K =  mimick_cols.shape

    # Set macroeconomic variables except of myp 
    var_exogcols = ts_var[var_exogcolnames].values
    T,M =  var_exogcols.shape

    # Starting value 
    output1_array = np.array(mimick_list[i].params)
    output2_array = np.array(var.params)
    output1_array_reshaped = output1_array.reshape(1, -1)
    output2_array_reshaped = output2_array.reshape(1, -1)
    startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)

    # GMM estimation
    var_mod = gmmvar(endog=zeros(var_exogcols.shape[0]), 
                  exog=var_exogcols, 
                  instrument=mimick_cols)
    var_fit = var_mod.fit(start_params=startVals, 
                        maxiter=0, 
                        inv_weights=eye(K+(M+1)*(M+2)), 
                        weights_method='hac', 
                        wargs={'maxlag':12},
                        # has_optimal_weights=False, 
                        optim_method='bfgs', 
                        optim_args={'gtol': 1e-15, 'maxiter': 10000})
    # print(var_fit.summary())

    # Append resuts
    gmmvar_list.append(var_fit)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


In [22]:
coeff_var_list = []
tvalues_var_list = []
pvalues_var_list = []
for i in range(len(gmmvar_list)):
    # Store results
    coeff_var = reshape(gmmvar_list[i].params[-((M+2)*(M+1)):],((M+2),(M+1)))
    tvalues_var = reshape(gmmvar_list[i].tvalues[-((M+2)*(M+1)):],((M+2),(M+1)))
    pvalues_var = reshape(gmmvar_list[i].pvalues[-((M+2)*(M+1)):],((M+2),(M+1)))
    coeff_var_list.append(coeff_var)
    tvalues_var_list.append(tvalues_var)
    pvalues_var_list.append(pvalues_var)

%store coeff_var_list
%store tvalues_var_list
%store pvalues_var_list

Stored 'coeff_var_list' (list)
Stored 'tvalues_var_list' (list)
Stored 'pvalues_var_list' (list)


Macroeconomic risk exposure

In [12]:
class gmmols(GMM):
    def momcond(self, params):
        global nbase
        fRets = self.exog
        pRets = self.endog
        mimick = self.instrument
        coeff = squeeze(array(params)) 

        # first stage: mimicking portfolio to compute myp
        mimick_endog = mimick[:,0]
        mimick_exog = mimick[:,1:]
        mimick_exog = sm.add_constant(mimick_exog)
        T,K = mimick_exog.shape
        mimick_exog = reshape(mimick_exog,(T,K))
        mimick_endog = reshape(mimick_endog,(T,1))
        mimick_coeff = coeff[:K]
        mimick_coeff = reshape(mimick_coeff,(K,1))
        mimick_err = (mimick_endog - mimick_exog @ mimick_coeff)
        moments1 = mimick_exog * kron(mimick_err,ones((1,K)))

        # compute myp
        myp = mimick_exog[:,1:(nbase+1)] @ mimick_coeff[1:(nbase+1)] 

        # second stage: time-series asset pricing model
        full_fRets = column_stack((ones((fRets.shape[0], 1)), myp, fRets))
        T,P = pRets.shape
        T,M = full_fRets.shape
        beta_params = params[K:]
        beta = squeeze(array(beta_params))
        beta = reshape(beta,(P,M))
        expectedRet = full_fRets @ beta.T
        e = pRets - expectedRet
        moments2 = kron(e,ones((1,M))) * tile(full_fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 

        return column_stack((moments1,moments2))

In [13]:
macro_exogcolnames = ['ui','dsv','ats','sts','fx']

beta_est_list = []
beta_params_list = [] 
beta_bse_list = [] 
beta_tvalues_list = []
beta_pvalues_list = [] 

for i in range(len(sdate)):
    beta_est = []
    beta_params = [] 
    beta_bse = [] 
    beta_tvalues = []
    beta_pvalues = [] 

    for index, port in enumerate(['bm','size','op','inv','mom']):

        ##############################################
        ######### Prepare the data ###################
        ##############################################
        ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
        ts4 = pd.merge(ts, ports_data, on='date', how='left')

        # Set the start and end dates of the analysis period
        ts4 = ts4.loc[(ts4.index >= sdate[i]) & (ts4.index <= edate[i])]

        nbase = 7
        mimick_colnames_sample = mimick_colnames
        if (ts4['slope_ex_mkt_87'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
                    nbase -= 1
        if (ts4['slope_ex_mkt_9602'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
                    nbase -= 1
        if (ts4['slope_ex_mkt_0709'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_0709' not in colname]
                    nbase -= 1
        mimick = ts4[mimick_colnames_sample].values

        T,K = mimick.shape
        exog_macro_factors = ts4[macro_exogcolnames].values
        T,M = exog_macro_factors.shape
        myp = ts4[[f'myp{i}']].values
        riskfree = ts4['rf'].values
        portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
        T,P = portfolios.shape
        excessRet = portfolios - reshape(riskfree,(T,1))

        ##############################################
        ######### Estimation #########################
        ##############################################

        # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
        betas = []
        for j in range(P):
            res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
            betas.append(res.params)
        betas = array(betas)

        output1_array = np.array(mimick_list[i].params)
        startingVals = np.concatenate([output1_array.flatten(), betas.flatten()])

        # Estimation    
        gmmols_mod = gmmols(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
        gmmols_fit = gmmols_mod.fit(start_params=startingVals, 
                                    maxiter=1, 
                                    inv_weights=eye(K+P*(M+2)),
                                    weights_method='hac', 
                                    wargs={'maxlag':12}, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})

        # Append results for a certain subsample
        beta_est.append(gmmols_fit)
        beta_params.append(gmmols_fit.params)
        beta_bse.append(gmmols_fit.bse)
        beta_tvalues.append(gmmols_fit.tvalues)
        beta_pvalues.append(gmmols_fit.pvalues)

    # Append results for all subsample
    beta_est_list.append(beta_est)
    beta_params_list.append(beta_params)
    beta_bse_list.append(beta_bse)
    beta_tvalues_list.append(beta_tvalues)
    beta_pvalues_list.append(beta_pvalues)

%store beta_params_list
%store beta_bse_list
%store beta_tvalues_list
%store beta_pvalues_list

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient e

In [14]:
macro_exogcolnames = ['ui','dsv','ats','sts','fx']

beta_est_list2 = []
beta_params_list2 = [] 
beta_bse_list2 = [] 
beta_tvalues_list2 = []
beta_pvalues_list2 = [] 

for i in range(len(sdate)):

        ##############################################
        ######### Prepare the data ###################
        ##############################################
        # Set the start and end dates of the analysis period
        ts4 = ts.loc[(ts.index >= sdate[i]) & (ts.index <= edate[i])]

        nbase = 7
        mimick_colnames_sample = mimick_colnames
        if (ts4['slope_ex_mkt_87'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
                    nbase -= 1
        if (ts4['slope_ex_mkt_9602'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
                    nbase -= 1
        if (ts4['slope_ex_mkt_0709'] == 0).all():
                    mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_0709' not in colname]
                    nbase -= 1
        mimick = ts4[mimick_colnames_sample].values

        T,K = mimick.shape
        exog_macro_factors = ts4[macro_exogcolnames].values
        T,M = exog_macro_factors.shape
        myp = ts4[[f'myp{i}']].values
        riskfree = ts4['rf'].values
        portfolios = ts4[['ex_mkt','hml','smb','rmw','cma','mom']].values        
        T,P = portfolios.shape
        excessRet = portfolios - reshape(riskfree,(T,1))

        ##############################################
        ######### Estimation #########################
        ##############################################

        # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
        betas = []
        for j in range(P):
            res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
            betas.append(res.params)
        betas = array(betas)

        output1_array = np.array(mimick_list[i].params)
        startingVals = np.concatenate([output1_array.flatten(), betas.flatten()])

        # Estimation    
        gmmols_mod = gmmols(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
        gmmols_fit = gmmols_mod.fit(start_params=startingVals, 
                                    maxiter=1, 
                                    inv_weights=eye(K+P*(M+2)),
                                    weights_method='hac', 
                                    wargs={'maxlag':12}, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})

        # Append results
        beta_est_list2.append(gmmols_fit)
        beta_params_list2.append(gmmols_fit.params)
        beta_bse_list2.append(gmmols_fit.bse)
        beta_tvalues_list2.append(gmmols_fit.tvalues)
        beta_pvalues_list2.append(gmmols_fit.pvalues)

%store beta_params_list2
%store beta_bse_list2
%store beta_tvalues_list2
%store beta_pvalues_list2

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Stored 'beta_params_list2' (list)
Stored 'beta_bse_list2' (list)
Stored 'beta_tvalues_list2' (list)
Stored 'beta_pvalues_list2' (list)
